In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import tqdm
from Libraries import Crawl

In [ ]:
# --- CONFIGURATION ---
BASE_URL = "https://vnexpress.net"
DATA_DIR = "../Database"
JSON_DIR = os.path.join(DATA_DIR, "JSON")
XLSX_DIR = os.path.join(DATA_DIR, "XLSX")
JSON_FILE = os.path.join(JSON_DIR, "vnexpress_articles.jsonl")
XLSX_FILE = os.path.join(XLSX_DIR, "vnexpress_articles.xlsx")
MIN_YEAR = 2020
MIN_WORDS = 300
MAX_WORDS = 1000
TARGET_ARTICLES_PER_SUBTYPE = 33
MAX_CONCURRENT_WORKERS = 10
REQUEST_TIMEOUT = 30

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

TYPE_DICT = {
    'thoi-su': ['chinh-tri', 'dan-sinh', 'giao-thong'],
    'the-gioi': ['tu-lieu', 'phan-tich', 'quan-su'],
    'kinh-doanh': ['quoc-te', 'doanh-nghiep', 'chung-khoan'],
    'khoa-hoc': ['tin-tuc', 'phat-minh', 'the-gioi-tu-nhien'],
    'giai-tri': ['gioi-sao', 'sach', 'phim', 'nhac'],
    'the-thao': ['bong-da', 'tennis', 'cac-mon-khac'],
    'giao-duc': ['tin-tuc', 'tuyen-sinh', 'du-hoc'],
    'suc-khoe': ['tin-tuc', 'cac-benh', 'song-khoe'],
}

VIETNAMESE_DAYS = {
    "Chủ nhật": "Sunday", "Thứ hai": "Monday", "Thứ ba": "Tuesday",
    "Thứ tư": "Wednesday", "Thứ năm": "Thursday", "Thứ sáu": "Friday",
    "Thứ bảy": "Saturday"
}

In [ ]:
def main():
    """Hàm chính điều phối việc crawl dữ liệu."""
    os.makedirs(JSON_DIR, exist_ok=True)
    os.makedirs(XLSX_DIR, exist_ok=True)
    
    existing_urls = Crawl.load_existing_urls(JSON_FILE)
    print(f"Đã tìm thấy {len(existing_urls)} URL đã được crawl.")

    with Crawl.create_session() as session:
        print("Bước 1: Thu thập tất cả URLs...")
        url_info_list = []
        for category, sub_categories in tqdm.tqdm(TYPE_DICT.items(), desc="Categories"):
            for sub_cat in sub_categories:
                urls = Crawl.get_article_urls(session, category, sub_cat)
                for url in urls:
                    url_info_list.append({'url': url, 'cat': category, 'sub': sub_cat})
        
        new_urls_to_scrape = [info for info in url_info_list if info['url'] not in existing_urls]
        
        print(f"Tổng cộng {len(url_info_list)} URL, trong đó có {len(new_urls_to_scrape)} URL mới.")
        if not new_urls_to_scrape:
            print("Không có bài báo mới nào để crawl.")
            return

        print("\nBước 2: Bắt đầu crawl chi tiết bài báo mới...")
        new_articles = []
        with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_WORKERS) as executor:
            future_to_url = {
                executor.submit(Crawl.scrape_article_details, session, info['url'], info['cat'], info['sub']): info 
                for info in new_urls_to_scrape
            }
            progress_bar = tqdm.tqdm(as_completed(future_to_url), total=len(new_urls_to_scrape), desc="Crawling Articles")
            for future in progress_bar:
                if result := future.result():
                    new_articles.append(result)

    if not new_articles:
        print("Không crawl được bài báo hợp lệ nào.")
        return

    print(f"\nBước 3: Lưu {len(new_articles)} bài báo mới vào file...")
    Crawl.json_export(new_articles)
    Crawl.xlsx_convert(JSON_FILE, XLSX_FILE)

if __name__ == "__main__":
    main()